# Wordle Solver

In [1]:
# import functions
from wordle_helpers import *

In [2]:
# help(main)

In [3]:
# Main function that returns a list of possible words
green_letters = 'T'
green_positions = [5,]
yellow_letters = 'ATOBA'
yellow_positions = [3,2,5,1,4,]
bad_letters = 'CRNESYLEP'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['abbot', 'about']


In [4]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      B      3
      A      2
      O      2
      T      2
      U      1


#### Good words:
SALET, CRANE, SHOUT, AUDIO, SLICE, SLICK, SPLIT, STUDY, INCUR, CHOIR, CROWN, CROWD, CRONE, POISE, OLIVE, DUNCE, BINGO, BISON, MIDST, SPOUT

In [5]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'ABPT'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

['bepat']


In [6]:
# help(score)

In [7]:
df = load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,Murilo,140
1,Barbara,60
2,Draw,141


In [8]:
# Update score
score_df = score()
score_df

,Names,Games_Won
0,Murilo,140
1,Barbara,61
2,Draw,141


In [9]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        140
Barbara         61
   Draw        141


In [10]:
# help(reset_score)

#### Set custom score

In [11]:
# help(set_score)

In [12]:
# # Set custom score
# df = set_score(m_score=140, b_score=60, draw_score=141)
# df

#### Reset score

In [13]:
## Reset score
# reset_score()